In [ ]:
#If you have already installed the module python-binance, please skip this step
!pip3 install python-binance
#Here is the official website if you need more information about python-binance: https://python-binance.readthedocs.io/en/latest/overview.html

In [1]:
import numpy as np
import pandas as pd
from Grid import Grid
from binance.client import Client

In [2]:
symbol = "BTCUSDT"
coin = "BTC"
timeFrame = "4h"
startDate = "2019-07-22"
endDate = "2022-07-22"
Binance = {"API":"MtK3YvsbesizJw7x8PzRKbcpnVDWE441aiCOjLn5qPBOX9ZLVBWCITgB8LwjBk7A", 
           "Secret":"YDGdHbK5dPkxI1noxuFLWhkE95N9jTJvwvX6uUBwAh0UhtMEzQXDhGCK2fi1aFDS"}

In [3]:
client = Client(Binance["API"], Binance["Secret"])

In [4]:
bars = client.get_historical_klines(symbol=f'{coin}USDT',interval=timeFrame,start_str=startDate, end_str = endDate)
test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
test_df = test_df.drop(["timestamp", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
test_df["open"] = pd.to_numeric(test_df["open"])
test_df["high"] = pd.to_numeric(test_df["high"])
test_df["low"] = pd.to_numeric(test_df["low"])
test_df["close"] = pd.to_numeric(test_df["close"])
test_df["volume"] = pd.to_numeric(test_df["volume"])

In [5]:
test_df

,open,high,low,close,volume,date
0,10590.15,10683.16,10552.00,10614.74,3019.256606,2019-07-22 00:00:00
1,10613.47,10642.66,10511.00,10603.21,4023.177652,2019-07-22 04:00:00
2,10603.39,10620.00,10407.30,10562.00,8024.209196,2019-07-22 08:00:00
3,10559.64,10588.00,10167.10,10238.43,11853.088262,2019-07-22 12:00:00
4,10240.81,10430.00,10100.00,10162.66,7855.224224,2019-07-22 16:00:00
...,...,...,...,...,...,...
6601,21090.22,21196.58,20706.50,20746.81,38069.879390,2022-07-26 12:00:00
6602,20746.81,21083.49,20734.79,20915.57,27918.741510,2022-07-26 16:00:00
6603,20911.55,21300.00,20809.53,21254.67,28947.946570,2022-07-26 20:00:00
6604,21254.67,21284.00,21042.53,21076.86,21881.708360,2022-07-27 00:00:00


In [6]:
parameters = {
    "grid_number" : 50,
    "equal_Diff_or_Ratio" : "DIFF", 
    "lowest_price" : 3000, 
    "highest_price" : 20000,
    "start_money" : 1000,
    "trading_fee_rate" : 0.002,
    "buy_unit" : 0.0001
}

myGrid = Grid(parameters)

In [7]:
window_size = 2000
profit = []
for i in range(len(test_df) - window_size):
    data = test_df.loc[i : i + window_size]
    data.reset_index(inplace =True)
    profit.append(myGrid.back_test_longOnly(data))

In [8]:
expected_return = np.average(profit)
risk_free_rate = 0.0204
standard_deviation = np.std(profit)

print("Sharpe ratio:", (expected_return - risk_free_rate) / standard_deviation)

Sharpe ratio: -44.16587934078796
